In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import wd

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo


In [17]:
def gen_pattern_replace_and_matcher_for_SingleMLP(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  key_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.embed_output_dim = shape_info[1]
          self.mlp = nn.Linear(shape_info[0],shape_info[1])


      def forward(self,x):
          x = self.embed(x).view(-1,self.embed_output_dim)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.redency_weight_len = self.embed_dim * redency_part_slice[1].stop
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.unredency_weight_len = unredency_weight.shape[1]
      self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
      self.redency_linear.weight.data.copy_(redency_weight)
      self.redency_linear.bias.data.copy_(linear_node_bias)
      self.redency_weight = torch.nn.Parameter(redency_weight.T,False)
      self.redency_bias = torch.nn.Parameter(linear_node_bias,False)
      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0],bias=False)
      self.unredency_linear.weight.data.copy_(unredency_weight)

      

    def forward(self,x):
      emb = self.embed(x)
      redency_part = emb[redency_part_slice] 
      unredency_part = emb[unredency_part_slice] 
      return  (torch.mm(redency_part.view(-1,self.redency_weight_len),self.redency_weight) + self.redency_bias) + self.unredency_linear(unredency_part.view(-1,self.unredency_weight_len))
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [20]:
def gen_pattern_replace_and_matcher_for_SingleMLP2(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  key_node_name,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[key_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()
          self.embed = torch.nn.Embedding(1, 1)
          self.embed_output_dim = shape_info[1]
          self.mlp = nn.Linear(shape_info[0],shape_info[1])


      def forward(self,x):
          x = self.embed(x).view(-1,self.embed_output_dim)
          return self.mlp(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  
  linear_node = node_map[pattern_env['mlp']]
  linear_node_module = utils.get_target_mod(traced,linear_node.target)
  linear_node_weight = linear_node_module.weight.data
  linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = embed_node_module
      self.embed_dim = self.embed.weight.data.shape[1]
      self.redency_weight_len = self.embed_dim * redency_part_slice[1].stop
      redency_weight = linear_node_weight[:,:self.redency_weight_len]
      unredency_weight = linear_node_weight[:,self.redency_weight_len:]
      self.unredency_weight_len = unredency_weight.shape[1]
      self.redency_linear = nn.Linear(redency_weight.shape[1],redency_weight.shape[0])
      self.redency_linear.weight.data.copy_(redency_weight)
      self.redency_linear.bias.data.copy_(linear_node_bias)
      print(linear_node_bias.shape)
      self.unredency_linear = nn.Linear(unredency_weight.shape[1],unredency_weight.shape[0],bias=False)
      self.unredency_linear.weight.data.copy_(unredency_weight)

      

    def forward(self,x):
      emb = self.embed(x)
      redency_part = emb[redency_part_slice] 
      unredency_part = emb[unredency_part_slice] 
      return self.redency_linear(redency_part.view(-1,self.redency_weight_len)) + self.unredency_linear(unredency_part.view(-1,self.unredency_weight_len))
      # return unredency_sum
    
  
  return pattern,ReplacementClass(),_match

In [18]:
def workload_wdl(num_field, prefix,dim = 64,l = [1024,512,256]):
  print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  wdl_model_ori = wd.WideAndDeepModel([100 for i in range(num_field)],dim,l,0.1)
  ori_traced = symbolic_trace(wdl_model_ori)
  
  wdl_model_modify = wd.WideAndDeepModel([100 for i in range(num_field)],dim,l,0.1)
  modify_traced = symbolic_trace(wdl_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_SingleMLP(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      "mlp_mlp_0")
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [10]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [21]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_wdl):
  def run(model):
    t = torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda()
    traced_model = torch.jit.trace(model.cuda(), t)
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    for i in range(100):
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)  
        start_event.record()

        with torch.no_grad():
          soutput = compiled_model(t)
        end_event.record()
        torch.cuda.synchronize()
        # 计算并打印函数执行所需的时间
        elapsed_time = start_event.elapsed_time(end_event)
        total_time.append(elapsed_time)
    print(calculate_mean_and_variance_manual(total_time[2:]))
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim)
  run(ori)
  run(modify)

In [22]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :1024
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/jit/_trace.py:1304: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 354 / 1024 (34.6%)
Greatest absolute difference: 0.004340022802352905 at index (321,) (up to 1e-05 allowed)
Greatest relative difference: 0.01702427872307491 at index (405,) (up to 1e-05 allowed)
  _check_trace(


(0.7082207336717722, 0.00016713861433740378)
(0.5356075106834879, 2.5122280018668383e-05)


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/jit/_trace.py:1304: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 401 / 1024 (39.2%)
Greatest absolute difference: 0.0049364566802978516 at index (986,) (up to 1e-05 allowed)
Greatest relative difference: 0.014523682212832036 at index (514,) (up to 1e-05 allowed)
  _check_trace(


In [9]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :2048
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 821 / 2048 (40.1%)
Greatest absolute difference: 0.003526061773300171 at index (400,) (up to 1e-05 allowed)
Greatest relative difference: 0.020118121448578465 at index (323,) (up to 1e-05 allowed)
  _check_trace(


(1.274575026667848, 0.0013852753364465068)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 766 / 2048 (37.4%)
Greatest absolute difference: 0.0030528903007507324 at index (1002,) (up to 1e-05 allowed)
Greatest relative difference: 0.01789087099515371 at index (442,) (up to 1e-05 allowed)
  _check_trace(


(0.9288963298408353, 0.0005980258677716522)


In [10]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1531 / 4096 (37.4%)
Greatest absolute difference: 0.0036971867084503174 at index (2340,) (up to 1e-05 allowed)
Greatest relative difference: 0.015987122149464333 at index (1652,) (up to 1e-05 allowed)
  _check_trace(


(2.3353884122809587, 0.006867955996979905)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1568 / 4096 (38.3%)
Greatest absolute difference: 0.003607451915740967 at index (2224,) (up to 1e-05 allowed)
Greatest relative difference: 0.017351435122971763 at index (1445,) (up to 1e-05 allowed)
  _check_trace(


(1.6373802496462453, 0.0021588837865913796)


In [11]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :8192
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3225 / 8192 (39.4%)
Greatest absolute difference: 0.005131959915161133 at index (4983,) (up to 1e-05 allowed)
Greatest relative difference: 0.01890614128463281 at index (2997,) (up to 1e-05 allowed)
  _check_trace(


(4.550021877094191, 0.013637985713781813)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3102 / 8192 (37.9%)
Greatest absolute difference: 0.004073679447174072 at index (6420,) (up to 1e-05 allowed)
Greatest relative difference: 0.01699567953640085 at index (4612,) (up to 1e-05 allowed)
  _check_trace(


(3.1609897053971583, 0.0052139720681881946)


In [12]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :1024
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 408 / 1024 (39.8%)
Greatest absolute difference: 0.003738224506378174 at index (384,) (up to 1e-05 allowed)
Greatest relative difference: 0.01507187595653089 at index (594,) (up to 1e-05 allowed)
  _check_trace(


(1.0397746958294694, 0.0022222831361790036)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 381 / 1024 (37.2%)
Greatest absolute difference: 0.003404855728149414 at index (355,) (up to 1e-05 allowed)
Greatest relative difference: 0.01435743868340365 at index (1012,) (up to 1e-05 allowed)
  _check_trace(


(0.7619487333054445, 0.0005158240993133667)


In [13]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :2048
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 771 / 2048 (37.6%)
Greatest absolute difference: 0.003822922706604004 at index (711,) (up to 1e-05 allowed)
Greatest relative difference: 0.016474492944113198 at index (495,) (up to 1e-05 allowed)
  _check_trace(


(2.136610287792829, 0.004868217012858205)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 803 / 2048 (39.2%)
Greatest absolute difference: 0.004019320011138916 at index (1135,) (up to 1e-05 allowed)
Greatest relative difference: 0.016755360902117704 at index (707,) (up to 1e-05 allowed)
  _check_trace(


(1.4683696323511553, 0.0013738032415744378)


In [14]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1578 / 4096 (38.5%)
Greatest absolute difference: 0.0038923025131225586 at index (4056,) (up to 1e-05 allowed)
Greatest relative difference: 0.018780772753050474 at index (1685,) (up to 1e-05 allowed)
  _check_trace(


(4.053875930455266, 0.017115892882119392)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1575 / 4096 (38.5%)
Greatest absolute difference: 0.004250705242156982 at index (2944,) (up to 1e-05 allowed)
Greatest relative difference: 0.018593955229629325 at index (1004,) (up to 1e-05 allowed)
  _check_trace(


(2.6455438380338707, 0.006635393316192698)


In [15]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 110, prefix: 50, batch :8192
now gen workload of wdl with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3102 / 8192 (37.9%)
Greatest absolute difference: 0.004059940576553345 at index (1395,) (up to 1e-05 allowed)
Greatest relative difference: 0.015738760153991754 at index (4954,) (up to 1e-05 allowed)
  _check_trace(


(8.158657988723444, 0.013924199894576897)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 3147 / 8192 (38.4%)
Greatest absolute difference: 0.0039237141609191895 at index (1392,) (up to 1e-05 allowed)
Greatest relative difference: 0.019279712669093355 at index (7434,) (up to 1e-05 allowed)
  _check_trace(


(5.152458453664974, 0.012604021616416886)


In [16]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 347 / 1024 (33.9%)
Greatest absolute difference: 0.00376969575881958 at index (952,) (up to 1e-05 allowed)
Greatest relative difference: 0.015377648105547495 at index (39,) (up to 1e-05 allowed)
  _check_trace(


(0.8695232661402955, 0.001978803543330226)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 348 / 1024 (34.0%)
Greatest absolute difference: 0.002809375524520874 at index (74,) (up to 1e-05 allowed)
Greatest relative difference: 0.012961231183960647 at index (81,) (up to 1e-05 allowed)
  _check_trace(


(0.4322377136167215, 9.854114752581504e-06)


In [17]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :2048
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 605 / 2048 (29.5%)
Greatest absolute difference: 0.0034074783325195312 at index (1630,) (up to 1e-05 allowed)
Greatest relative difference: 0.01421109490026618 at index (1366,) (up to 1e-05 allowed)
  _check_trace(


(1.7053355136696173, 0.004141560647210402)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 667 / 2048 (32.6%)
Greatest absolute difference: 0.0027750134468078613 at index (447,) (up to 1e-05 allowed)
Greatest relative difference: 0.016560066393882796 at index (1119,) (up to 1e-05 allowed)
  _check_trace(


(0.7231346928343481, 0.00019961166332945263)


In [18]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1242 / 4096 (30.3%)
Greatest absolute difference: 0.003514111042022705 at index (1156,) (up to 1e-05 allowed)
Greatest relative difference: 0.02100075394976343 at index (1721,) (up to 1e-05 allowed)
  _check_trace(


(3.3143252231636824, 0.008442764811563206)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1336 / 4096 (32.6%)
Greatest absolute difference: 0.0037213563919067383 at index (2395,) (up to 1e-05 allowed)
Greatest relative difference: 0.016243217071078984 at index (3360,) (up to 1e-05 allowed)
  _check_trace(


(1.268198859934904, 0.0012240933643346147)


In [19]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :8192
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2595 / 8192 (31.7%)
Greatest absolute difference: 0.005005359649658203 at index (719,) (up to 1e-05 allowed)
Greatest relative difference: 0.02224574816144285 at index (7907,) (up to 1e-05 allowed)
  _check_trace(


(6.637694684826598, 0.010420738814504397)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2556 / 8192 (31.2%)
Greatest absolute difference: 0.0038712620735168457 at index (3647,) (up to 1e-05 allowed)
Greatest relative difference: 0.01745092021859272 at index (1271,) (up to 1e-05 allowed)
  _check_trace(


(2.429043275969369, 0.0025215345660416005)


In [20]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :1024
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 331 / 1024 (32.3%)
Greatest absolute difference: 0.003772139549255371 at index (424,) (up to 1e-05 allowed)
Greatest relative difference: 0.015179871624202338 at index (692,) (up to 1e-05 allowed)
  _check_trace(


(1.530930617634131, 0.006148601322350067)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 329 / 1024 (32.1%)
Greatest absolute difference: 0.0033292770385742188 at index (537,) (up to 1e-05 allowed)
Greatest relative difference: 0.01270873856483735 at index (825,) (up to 1e-05 allowed)
  _check_trace(


(0.6163885757631186, 4.811531890003044e-05)


In [21]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :2048
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 648 / 2048 (31.6%)
Greatest absolute difference: 0.0032529830932617188 at index (483,) (up to 1e-05 allowed)
Greatest relative difference: 0.01938085121590814 at index (5,) (up to 1e-05 allowed)
  _check_trace(


(3.0521632822192446, 0.009117670969901865)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 653 / 2048 (31.9%)
Greatest absolute difference: 0.005194485187530518 at index (315,) (up to 1e-05 allowed)
Greatest relative difference: 0.015460032447406432 at index (183,) (up to 1e-05 allowed)
  _check_trace(


(1.0825841597148351, 0.00015699528787089487)


In [22]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1280 / 4096 (31.2%)
Greatest absolute difference: 0.004723548889160156 at index (431,) (up to 1e-05 allowed)
Greatest relative difference: 0.018204968469031482 at index (2530,) (up to 1e-05 allowed)
  _check_trace(


(6.018022522634389, 0.011895547523401643)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1291 / 4096 (31.5%)
Greatest absolute difference: 0.003751814365386963 at index (1480,) (up to 1e-05 allowed)
Greatest relative difference: 0.021779289118477027 at index (3186,) (up to 1e-05 allowed)
  _check_trace(


(1.954504815899596, 0.07751386293767097)


In [23]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 170, prefix: 145, batch :8192
now gen workload of wdl with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2611 / 8192 (31.9%)
Greatest absolute difference: 0.0036445260047912598 at index (3159,) (up to 1e-05 allowed)
Greatest relative difference: 0.018203432953204188 at index (6651,) (up to 1e-05 allowed)
  _check_trace(


(11.749561484979123, 0.057158873852379104)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2575 / 8192 (31.4%)
Greatest absolute difference: 0.0044355690479278564 at index (6262,) (up to 1e-05 allowed)
Greatest relative difference: 0.017009800180718827 at index (7267,) (up to 1e-05 allowed)
  _check_trace(


(3.66764636672273, 0.004347398055342344)


In [10]:
def gen_and_test_profile(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_wdl):
  def run(model):
    t = torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda()
    traced_model = torch.jit.trace(model.cuda(), t)
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []

    with profile(activities=[torch.profiler.ProfilerActivity.CUDA,ProfilerActivity.CPU], record_shapes=True,
    profile_memory=True,  # 记录内存使用情况
    with_stack=True,  # 记录代码堆栈信息
    with_flops=True,  # 记录FLOPs
    on_trace_ready=torch.profiler.tensorboard_trace_handler('./log_wdl_gpu')  
                   
                   
                   
                   ) as prof:
        with record_function("model_inference"):
          for i in range(6):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              torch.cuda.synchronize()
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time[2:]))
  print(f"now gen workload of fm with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim)
  run(ori)
  run(modify)

In [30]:
gen_and_test_profile(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :1024
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 397 / 1024 (38.8%)
Greatest absolute difference: 0.00351637601852417 at index (101,) (up to 1e-05 allowed)
Greatest relative difference: 0.015655356434982094 at index (570,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:28 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 17:00:28 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:28 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4271736145019531, 0.3091613791639247)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 382 / 1024 (37.3%)
Greatest absolute difference: 0.00342714786529541 at index (506,) (up to 1e-05 allowed)
Greatest relative difference: 0.01801913079204218 at index (499,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:28 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.1718273162841797, 0.0008579661425756058)


STAGE:2025-02-08 17:00:29 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:29 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [31]:
gen_and_test_profile(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :2048
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 808 / 2048 (39.5%)
Greatest absolute difference: 0.003586709499359131 at index (1207,) (up to 1e-05 allowed)
Greatest relative difference: 0.021112000735237994 at index (1613,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:33 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 17:00:33 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:33 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.4615654945373535, 0.00019691626107487537)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 756 / 2048 (36.9%)
Greatest absolute difference: 0.0037312209606170654 at index (520,) (up to 1e-05 allowed)
Greatest relative difference: 0.017519353117347806 at index (1925,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:33 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 17:00:33 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:33 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.2483596801757812, 0.00015691625776526053)


In [32]:
gen_and_test_profile(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1593 / 4096 (38.9%)
Greatest absolute difference: 0.0035280585289001465 at index (2452,) (up to 1e-05 allowed)
Greatest relative difference: 0.013900145652629179 at index (245,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:36 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 17:00:36 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:36 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.3398399353027344, 0.0007296705462067621)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1537 / 4096 (37.5%)
Greatest absolute difference: 0.003987699747085571 at index (4022,) (up to 1e-05 allowed)
Greatest relative difference: 0.022189694799979495 at index (2860,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:36 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.844644546508789, 0.0002162892087653745)


STAGE:2025-02-08 17:00:37 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:37 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [33]:
gen_and_test_profile(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 323 / 1024 (31.5%)
Greatest absolute difference: 0.003476440906524658 at index (813,) (up to 1e-05 allowed)
Greatest relative difference: 0.024741860176785636 at index (891,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:39 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 17:00:40 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:40 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.0885000228881836, 0.0003667111059257877)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 312 / 1024 (30.5%)
Greatest absolute difference: 0.0029533803462982178 at index (709,) (up to 1e-05 allowed)
Greatest relative difference: 0.01706106898966247 at index (222,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:40 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.1813640594482422, 6.602363100682851e-05)


STAGE:2025-02-08 17:00:40 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:40 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [34]:
gen_and_test_profile(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :2048
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 671 / 2048 (32.8%)
Greatest absolute difference: 0.0030243396759033203 at index (1214,) (up to 1e-05 allowed)
Greatest relative difference: 0.0148026098853159 at index (1287,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:45 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 17:00:45 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:45 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.7979145050048828, 8.628830983070657e-05)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 640 / 2048 (31.2%)
Greatest absolute difference: 0.0031096935272216797 at index (373,) (up to 1e-05 allowed)
Greatest relative difference: 0.014267576718779772 at index (1312,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:46 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.150965690612793, 5.3447024583874736e-05)


STAGE:2025-02-08 17:00:46 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:46 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [35]:
gen_and_test_profile(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 170, prefix: 145, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1337 / 4096 (32.6%)
Greatest absolute difference: 0.004056990146636963 at index (496,) (up to 1e-05 allowed)
Greatest relative difference: 0.016664702564836403 at index (1278,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:52 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 17:00:53 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:53 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.1297802925109863, 0.00019111823235107295)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1301 / 4096 (31.8%)
Greatest absolute difference: 0.0035280585289001465 at index (2582,) (up to 1e-05 allowed)
Greatest relative difference: 0.018102606720612868 at index (2859,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:00:53 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.4485716819763184, 0.00010019718388321053)


STAGE:2025-02-08 17:00:53 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:00:53 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [23]:
gen_and_test_profile(num_field = 100,prefix = 10, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 10, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 100, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1645 / 4096 (40.2%)
Greatest absolute difference: 0.004039347171783447 at index (3779,) (up to 1e-05 allowed)
Greatest relative difference: 0.015320448869185673 at index (509,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:11 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:35:11 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:11 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.5043954849243164, 0.00010898304481088417)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1670 / 4096 (40.8%)
Greatest absolute difference: 0.0038939714431762695 at index (1140,) (up to 1e-05 allowed)
Greatest relative difference: 0.014898968308992545 at index (2214,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:11 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(3.4846067428588867, 9.906386821967317e-05)


STAGE:2025-02-08 16:35:11 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:11 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [24]:
gen_and_test_profile(num_field = 100,prefix = 20, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 20, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 100, prefix: 20


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1627 / 4096 (39.7%)
Greatest absolute difference: 0.003161787986755371 at index (3962,) (up to 1e-05 allowed)
Greatest relative difference: 0.01858619078893934 at index (34,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:14 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:35:14 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:14 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.4917593002319336, 0.00010122391813638387)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1585 / 4096 (38.7%)
Greatest absolute difference: 0.0029675066471099854 at index (3171,) (up to 1e-05 allowed)
Greatest relative difference: 0.01546040252254922 at index (3158,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:14 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(3.236234188079834, 0.00010832579278030607)


STAGE:2025-02-08 16:35:14 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:14 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [25]:
gen_and_test_profile(num_field = 100,prefix = 30, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 30, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 100, prefix: 30


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1577 / 4096 (38.5%)
Greatest absolute difference: 0.0035378336906433105 at index (3111,) (up to 1e-05 allowed)
Greatest relative difference: 0.01388625649457763 at index (3706,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:17 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:35:17 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:17 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.4891366958618164, 6.015454800945008e-05)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1731 / 4096 (42.3%)
Greatest absolute difference: 0.005003035068511963 at index (3446,) (up to 1e-05 allowed)
Greatest relative difference: 0.018882474936921155 at index (3506,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:17 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(2.9845237731933594, 5.0249582272954285e-05)


STAGE:2025-02-08 16:35:17 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:17 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [26]:
gen_and_test_profile(num_field = 100,prefix = 40, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 40, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 100, prefix: 40


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1628 / 4096 (39.7%)
Greatest absolute difference: 0.003016531467437744 at index (531,) (up to 1e-05 allowed)
Greatest relative difference: 0.01382078297966005 at index (2021,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:21 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:35:21 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:21 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.5030245780944824, 0.00011543122013790708)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1531 / 4096 (37.4%)
Greatest absolute difference: 0.004174232482910156 at index (3570,) (up to 1e-05 allowed)
Greatest relative difference: 0.014275693731105956 at index (52,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:21 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:35:21 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:21 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.6828646659851074, 0.00012168399621259596)


In [27]:
gen_and_test_profile(num_field = 100,prefix = 50, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 50, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 100, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1666 / 4096 (40.7%)
Greatest absolute difference: 0.0038893520832061768 at index (193,) (up to 1e-05 allowed)
Greatest relative difference: 0.014540952131589681 at index (1602,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:24 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:35:24 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:24 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.515779972076416, 0.00012034817586936697)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1663 / 4096 (40.6%)
Greatest absolute difference: 0.0042507946491241455 at index (1654,) (up to 1e-05 allowed)
Greatest relative difference: 0.0169568962135126 at index (2207,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:24 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(2.4830102920532227, 0.001049684783538396)


STAGE:2025-02-08 16:35:24 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:24 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [28]:
gen_and_test_profile(num_field = 100,prefix = 60, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 60, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 100, prefix: 60


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1665 / 4096 (40.6%)
Greatest absolute difference: 0.004527032375335693 at index (3817,) (up to 1e-05 allowed)
Greatest relative difference: 0.01509551117236396 at index (287,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:26 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:35:27 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:27 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.5083889961242676, 2.794919851112354e-05)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1623 / 4096 (39.6%)
Greatest absolute difference: 0.0035148262977600098 at index (3170,) (up to 1e-05 allowed)
Greatest relative difference: 0.019745320223508236 at index (1337,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:35:27 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(2.202928066253662, 4.136424536227423e-05)


STAGE:2025-02-08 16:35:27 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:35:27 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [22]:
gen_and_test_profile(num_field = 100,prefix = 70, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 70, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 100, prefix: 70


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1675 / 4096 (40.9%)
Greatest absolute difference: 0.003936409950256348 at index (3007,) (up to 1e-05 allowed)
Greatest relative difference: 0.01817842731617518 at index (607,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:34:25 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:34:25 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:34:25 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.5118460655212402, 0.00033507419061606925)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1659 / 4096 (40.5%)
Greatest absolute difference: 0.005254864692687988 at index (1994,) (up to 1e-05 allowed)
Greatest relative difference: 0.018939759134265932 at index (12,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:34:25 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up


(1.9313693046569824, 6.728484436280269e-05)


STAGE:2025-02-08 16:34:26 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:34:26 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [21]:
gen_and_test_profile(num_field = 100,prefix = 80, batch = 4096, dim = 64)

now gen workload of fm with config: dim: 64, num_field: 100, prefix: 80, batch :4096
now gen workload of wdl with config: dim: 64, num_field: 100, prefix: 80


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1654 / 4096 (40.4%)
Greatest absolute difference: 0.003789186477661133 at index (2929,) (up to 1e-05 allowed)
Greatest relative difference: 0.016508535029974104 at index (3595,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:34:15 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:34:16 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:34:16 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3.495454788208008, 0.00019065282685915008)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1693 / 4096 (41.3%)
Greatest absolute difference: 0.004492580890655518 at index (2989,) (up to 1e-05 allowed)
Greatest relative difference: 0.0165574753223257 at index (783,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 16:34:16 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 16:34:16 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 16:34:16 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.6617178916931152, 3.340616672176111e-05)


In [36]:
gen_and_test_profile(num_field = 100,prefix = 80, batch = 4096, dim = 32)

now gen workload of fm with config: dim: 32, num_field: 100, prefix: 80, batch :4096
now gen workload of wdl with config: dim: 32, num_field: 100, prefix: 80


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1585 / 4096 (38.7%)
Greatest absolute difference: 0.004495680332183838 at index (2364,) (up to 1e-05 allowed)
Greatest relative difference: 0.019290215686929305 at index (1330,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:21:03 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 17:21:03 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:21:03 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2.1816492080688477, 5.2935433814127464e-05)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1653 / 4096 (40.4%)
Greatest absolute difference: 0.003726184368133545 at index (454,) (up to 1e-05 allowed)
Greatest relative difference: 0.015649135351320138 at index (1830,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-08 17:21:03 4057170:4057170 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-08 17:21:03 4057170:4057170 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-08 17:21:03 4057170:4057170 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1.3285279273986816, 0.001293852136541318)
